In [15]:
from pprint import pprint

import numpy as np

from sklearn.model_selection import train_test_split
from skimage.transform import resize

import matplotlib.pyplot as plt

from keras import Sequential
import keras.layers as layers
from keras.regularizers import l2
from keras.utils import to_categorical

from sklearn.metrics import classification_report

import tensorflow as tf

from pathlib import Path

# Preprocess data

In [3]:
# loading images
dog_directory = Path("images")

breeds = []
for breed in dog_directory.iterdir():
    # count number of images in each breed
    if breed.is_dir():
        breed_count = len(list(breed.glob("*.jpg")))
        breeds.append((breed.name.split("-", 1)[-1], breed_count))

breeds.sort(key=lambda x: x[1], reverse=True)
breeds = [breed[0] for breed in breeds[:10]]
print(breeds)

['Maltese_dog', 'Afghan_hound', 'Scottish_deerhound', 'Pomeranian', 'Irish_wolfhound', 'Bernese_mountain_dog', 'Samoyed', 'Shih-Tzu', 'Great_Pyrenees', 'Leonberg']


In [93]:
# load images
X = []
y = []
width, height = 128, 128 # image resolution

for breed in dog_directory.iterdir():
    breed_name = breed.name.split("-", 1)[-1]
    for i, image_path in enumerate(breed.glob("*.jpg")):
        if breed_name not in breeds:
            continue
        image = plt.imread(image_path)
        image = np.rot90(image, 90*(i+1)) # rotate image
        image = resize(image, (width, height, 3))
        X.append(image)
        y.append(breeds.index(breed_name))
        if i >= 139:  # do 140 images each
            break
    print(f"Loaded {len(X)} images for {breed_name}")

X = np.array(X)
y = np.array(y)

print(f"{X=}, {y=}")

Loaded 0 images for silky_terrier
Loaded 140 images for Scottish_deerhound
Loaded 140 images for Chesapeake_Bay_retriever
Loaded 140 images for Ibizan_hound
Loaded 140 images for wire-haired_fox_terrier
Loaded 140 images for Saluki
Loaded 140 images for cocker_spaniel
Loaded 140 images for schipperke
Loaded 140 images for borzoi
Loaded 140 images for Pembroke
Loaded 140 images for komondor
Loaded 140 images for Staffordshire_bullterrier
Loaded 140 images for standard_poodle
Loaded 140 images for Eskimo_dog
Loaded 140 images for English_foxhound
Loaded 140 images for golden_retriever
Loaded 140 images for Sealyham_terrier
Loaded 140 images for Japanese_spaniel
Loaded 140 images for .DS_Store
Loaded 140 images for miniature_schnauzer
Loaded 140 images for malamute
Loaded 140 images for malinois
Loaded 140 images for Pekinese
Loaded 140 images for giant_schnauzer
Loaded 140 images for Mexican_hairless
Loaded 140 images for Doberman
Loaded 140 images for standard_schnauzer
Loaded 140 image

In [94]:
# label data
labels = to_categorical(y, num_classes=10)
labels = np.array(labels, dtype="int32")
print(labels)                                                           

[[0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


# Splitting data

In [95]:
# Split data into training and test samples
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, labels,   
    test_size = 0.07, random_state=10, shuffle=True
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, 
    test_size = 0.07, random_state=10, shuffle=True
)

# Scale data
X_train = X_train/255
X_val = X_val/255
X_test = X_test/255

In [86]:
# Check to make sure everything is as expected
print('X_train:' + str(X_train.shape))
print('y_train:' + str(y_train.shape))
print('X_val: \t'  + str(X_val.shape))
print('y_val: \t'  + str(y_val.shape))
print('X_test: '  + str(X_test.shape))
print('y_test: '  + str(y_test.shape))

X_train:(1209, 128, 128, 3)
y_train:(1209, 10)
X_val: 	(92, 128, 128, 3)
y_val: 	(92, 10)
X_test: (99, 128, 128, 3)
y_test: (99, 10)


# Functions

In [ ]:
def train_model(activation="sigmoid", stride=(4, 4), epoch=100):
    model = Sequential([
    layers.Input(shape=(height, width, 3)),
    layers.Conv2D(256, 3, padding="same", activation=activation),
    layers.MaxPooling2D(strides=stride), # default: stride=(2,2)
    layers.Conv2D(512, 3, padding="same", activation=activation),
    layers.MaxPooling2D(strides=stride),
    layers.Conv2D(724, 3, padding="same", activation=activation),
    # layers.MaxPooling2D(strides=stride),
    # layers.Conv2D(128, 3, padding="same", activation=activation),
    # layers.MaxPooling2D(strides=stride),
    # layers.Conv2D(256, 3, padding="same", activation=activation),
    layers.Flatten(),
    layers.Dense(1024, activation=activation),
    layers.Dense(10, activation="softmax")
    ])
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    history = model.fit(X_train, y_train, epochs = epoch, validation_data=(X_val, y_val))
    model.evaluate(X_val, y_val)
    return history, model

def class_report(history, model):
    print(f"train_acc={np.mean(history.history["accuracy"])}")
    print(f"val_acc={np.mean(history.history["val_accuracy"])}")
    res = model.predict(X_test)
    y_test = np.array(y_test, dtype="int32")

    p, q = [], []

    for i in range(len(res)):
        p.append(np.argmax(res[i]))
        pred = breeds[np.argmax(res[i])]
        q.append(np.argmax(y_test[i]))

        actual = breeds[np.argmax(y_test[i])]
        # print(f"Predicted: {pred}, Actual: {actual}")

    # top_breeds = ["French_bulldog", "golden_retriever", 
    # "German_shepherd", "standard_poodle", "Samoyed", 
    # "French_bulldog", "beagle", "Rottweiler"]
    print(classification_report(p, q))

SyntaxError: f-string: unmatched '[' (430631068.py, line 23)

# Test

In [ ]:
# Sigmoid (control)
history, model = train_model()
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Sigmoid Activation Function)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

In [ ]:
# Hidden Layers
# 5 Layers
history, model = train_model()
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Five Hidden Layers)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

# 3 Layers
history, model = train_model()
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Three Hidden Layers)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

In [ ]:
# Activation Functions
# ReLU
history, model = train_model("leaky_relu")
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Leaky ReLU Activation Function)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

# Tanh
history, model = train_model("tanh")
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Tanh Activation Function)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

In [ ]:
# Iterations
# 400 Iterations
history, model = train_model("sigmoid", (4, 4), 200)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (400 Iterations)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

# 200 Iterations
history, model = train_model("sigmoid", (4, 4), 100)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (200 Iterations)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

In [ ]:
# Strides
# (3, 3))
history, model= train_model("sigmoid", (3, 3), 100)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (3x3 Strides)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

# (4, 4)
history, model = train_model("sigmoid", (4, 4), 100)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (4x4 Strides)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

# (5, 5)
history, model = train_model("sigmoid", (5, 5), 100)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (5x5 Strides)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

In [97]:
# Rotate image
history, model = train_model("sigmoid", (5, 5), 100)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Rotating Image)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

Epoch 1/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.0832 - loss: 8.0036 - precision_31: 0.0725 - recall_31: 0.0414 - val_accuracy: 0.0761 - val_loss: 2.7753 - val_precision_31: 0.0000e+00 - val_recall_31: 0.0000e+00
Epoch 2/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.1010 - loss: 2.4246 - precision_31: 0.0000e+00 - recall_31: 0.0000e+00 - val_accuracy: 0.0543 - val_loss: 2.3868 - val_precision_31: 0.0000e+00 - val_recall_31: 0.0000e+00
Epoch 3/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.0857 - loss: 2.3465 - precision_31: 0.0000e+00 - recall_31: 0.0000e+00 - val_accuracy: 0.1196 - val_loss: 2.3103 - val_precision_31: 0.0000e+00 - val_recall_31: 0.0000e+00
Epoch 4/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.0678 - loss: 2.3510 - precision_31: 0.0000e+00 - recall_31: 0.0000e+00 - val_accuracy: 0.1304 - val_loss: 2.3223 - val_precision_31: 0.0000e+00 - val_recall_31: 0.0000e+00
Epoch 5/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accurac

In [92]:
# # Normalized
# history, model = train_model("sigmoid", 100)
# plt.plot(history.history["loss"], label="Training Error")
# plt.plot(history.history["val_loss"], label="Validation Error")
# plt.legend(loc="upper right")
# plt.title("Error vs Epochs (Normalization)")
# plt.xlabel("Epochs")
# plt.ylabel("Loss")
# class_report(history, model)

# # Not Normalized
# history, model = train_model("sigmoid", 100)
# plt.plot(history.history["loss"], label="Training Error")
# plt.plot(history.history["val_loss"], label="Validation Error")
# plt.legend(loc="upper right")
# plt.title("Error vs Epochs (No Normalization)")
# plt.xlabel("Epochs")
# plt.ylabel("Loss")
# class_report(history, model)

In [ ]:
# Regularizers
def train_model_reg(l, activation="sigmoid", stride=(4,4), epoch=100):
    model = Sequential([
    layers.Input(shape=(height, width, 3)),
    layers.Conv2D(256, 3, padding="same", activation=activation, kernel_regularizer=regularizers.l2(l)),
    layers.MaxPooling2D(strides=stride), # default: stride=(2,2)
    layers.Conv2D(512, 3, padding="same", activation=activation, kernel_regularizer=regularizers.l2(l)),
    layers.MaxPooling2D(strides=stride),
    layers.Conv2D(724, 3, padding="same", activation=activation, kernel_regularizer=regularizers.l2(l)),
    # layers.MaxPooling2D(strides=stride),
    # layers.Conv2D(128, 3, padding="same", activation=activation),
    # layers.MaxPooling2D(strides=stride),
    # layers.Conv2D(256, 3, padding="same", activation=activation),
    layers.Flatten(),
    layers.Dense(1024, activation=activation, kernel_regularizer=regularizers.l2(l)),
    layers.Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(l))
    ])
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    history = model.fit(X_train, y_train, epochs = epoch, validation_data=(X_val, y_val))
    model.evaluate(X_val, y_val)
    return history

# Lambda = 0.0001
history, model = history = train_model(0.0001)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Lambda = 0.0001)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

# Lambda = 0.001
history, model = train_model(0.001)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Lambda = 0.001)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

# Lambda = 0.01
history, model = train_model(0.01)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Lambda = 0.01)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

# Lambda = 1.0
history, model = train_model(1.0)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Lambda = 1.0)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

# Lambda = 10.0
history, model = train_model(10.0)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Lambda = 10.0)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)

# Lambda = 100.0
history, model = train_model(100.0)
plt.plot(history.history["loss"], label="Training Error")
plt.plot(history.history["val_loss"], label="Validation Error")
plt.legend(loc="upper right")
plt.title("Error vs Epochs (Lambda = 100.0)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
class_report(history, model)